# A numeric approach to the Monty Hall problem

The Monty Hall problem is famous in the world of probability


> You're playing a gameshow, and the host asks you to choose one of three doors.
>
> Behind one door is a car, and the other two doors have goats. You win whatever you choose!
>
> Once you have chosen a door, the host
> 1. Removes one of doors that you didn't choose (but he will never remove the car), then
> 2. Gives you the option to change your choice of door.
>
> Should you change? Does it matter?

Let's set up the problem

In [1]:
GOAT = 'goat'
CAR = 'car'
DOORS = (0, 1, 2)
AVAILABLE_PRIZES = (GOAT, GOAT, CAR)

## Let's Play

First, we need to arrange our car and goats behind some doors

In [2]:
import random

def make_prizes():
    return random.sample(AVAILABLE_PRIZES, 3)

prizes = make_prizes()

No, I'm not going to *show you them!!!!*  That's not how this game works!  

But it looks something like `[GOAT, GOAT, CAR]` or some permutation of that list.


And we need a way to know if you win or not!

In [3]:
def its_a_carrrrr(door, prizes):
    return prizes[door] is CAR

So now it's time to choose a door

In [4]:
def choose_door():
    return random.choice(DOORS)

first_choice = choose_door()
first_choice

2

If you don't like the idea of Python choosing randomly for you, please be my guest and set

`first_choice = 0` (or 1 or 2)

## The Great Removal
Then the host has to remove a door.

Implementing this code really helped me to understand the nuance in the Monty Hall Problem.

It has to do with how the host decides which door to remove...

In [5]:
def is_removable(door, first_choice, prizes):
    '''
    Remember the host isn't allowed to remove 
     - the door with the car behind it, or
     - the door you chose at the start.
     
     (and maybe that's the same door!)
     '''
    return door is not first_choice \
       and prizes[door] is not CAR

def find_removable_doors(first_choice, prizes):
    ''' 
    There's something interesting here...
    
    When player chooses the car first go, 
        the host has the *choice* of two goats he could remove.
        
    However, when the player chooses a goat first go,
        the host has no choice, he *must* remove the other goat
    '''
    return [
        door for door in DOORS
        if is_removable(door, first_choice, prizes)
    ]

def remove_a_door(first_choice, prizes):
    '''
    So the host's random choice of removable doors isn't always random.
    Two thirds of the time (when the player chose a goat on their first choice), 
    the host removes the only door he's allowed to remove (the other goat).
    '''
    removable_doors = find_removable_doors(first_choice, prizes)
    return random.choice(removable_doors)

def remaining(first_choice, removed_door):
    return next(door for door in DOORS
                if door is not first_choice
                and door is not removed_door)


*Interesting that the host doesn't always have a choice which door he removes... huh...*

Which door did he remove this time?

In [6]:
removed_door = remove_a_door(first_choice, prizes)
removed_door

0

Which leaves one remaining door - the door that is not chosen and not removed.

In [7]:
the_other_door = remaining(first_choice, removed_door)
the_other_door

1

So now you have the choice.


In [8]:
def announce_outcome(prizes, door):
    if its_a_carrrrr(door, prizes):
        print(f'Wooohoooooo you won a {CAR}!!!!!')
    else:
        print(f"It's a {GOAT} - Happy Grazing <3")

print('Did you stay with your first choice?', end=' ')
announce_outcome(prizes, door=first_choice)

print('Or did you change your mind?', end=' ')
announce_outcome(prizes, door=the_other_door)

Did you stay with your first choice? It's a goat - Happy Grazing <3
Or did you change your mind? Wooohoooooo you won a car!!!!!


## The Monty Hallway

So now we played the game once.

To really test whether it's better to stay with your first choice or not, we'd better play a few more times.
 
First, let's make a function to play the whole game

In [9]:
def play_monty_hall(change):
    prizes = make_prizes()
    first_choice = choose_door()
    removed_door = remove_a_door(first_choice, prizes)
    the_other_door = remaining(first_choice, removed_door)
    final_choice = the_other_door if change else first_choice
    return its_a_carrrrr(final_choice, prizes)

Will you win this time without changing?

In [10]:
play_monty_hall(change=False)

False

And how about now, changing your mind?

In [11]:
play_monty_hall(change=True)

True

Okay so it works.

Time to test the strategies.

How many times should we play with each strategy?

In [12]:
n_trials = 10000

Let's play!

In [13]:
wins_without_change = sum(play_monty_hall(change=False) for _ in range(n_trials))
wins_with_change = sum(play_monty_hall(change=True) for _ in range(n_trials))

In [14]:
print(f'Wins without change {wins_without_change}, {100*wins_without_change/n_trials}%')
print(f'Wins with change {wins_with_change}, {100*wins_with_change/n_trials}%')

Wins without change 3394, 33.94%
Wins with change 6637, 66.37%


So it seems like changing is a much better strategy!

Here's the final code

In [15]:
import random


GOAT = 'goat'
CAR = 'car'
DOORS = (0, 1, 2)
AVAILABLE_PRIZES = (GOAT, GOAT, CAR)


def make_prizes():
    return random.sample(AVAILABLE_PRIZES, 3)

def choose_door():
    return random.choice(DOORS)

def is_removable(door, first_choice, prizes):
    return door is not first_choice \
       and prizes[door] is not CAR

def remove_a_door(first_choice, prizes):
    removable_doors = [door for door in DOORS if is_removable(door, first_choice, prizes)]
    return random.choice(removable_doors)

def remaining(first_choice, removed_door):
    return next(door for door in DOORS
                if door is not first_choice
                and door is not removed_door)

def its_a_carrrrr(door, prizes):
    return prizes[door] is CAR

def play_monty_hall(change):
    prizes = make_prizes()
    first_choice = choose_door()
    removed_door = remove_a_door(first_choice, prizes)
    the_other_door = remaining(first_choice, removed_door)
    final_choice = the_other_door if change else first_choice
    return its_a_carrrrr(final_choice, prizes)

def compare_strategies(n_trials=1000):
    wins_without_change = sum(play_monty_hall(change=False) for _ in range(n_trials))
    wins_with_change = sum(play_monty_hall(change=True) for _ in range(n_trials))
    return (
        f'Wins without change {wins_without_change}, {100*wins_without_change/n_trials}%'
        '\n'
        f'Wins with change {wins_with_change}, {100*wins_with_change/n_trials}%'
    )
    
print(compare_strategies(100000))

Wins without change 33165, 33.165%
Wins with change 66677, 66.677%
